In [160]:
import redis
import hiredis
import ranking
import os
import evaluate
import conn
from rq import Queue
import ast
from tournament import Team, Game
import pprint
import random
import statistics as stats
import math
import mock
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint
redis_host = 'localhost'
redis_port = '6379'
wait_time = '600'
r = redis.StrictRedis(host=redis_host, port=redis_port, db=0)
session = {'s_ind':1}
s_ind = session['s_ind']
dc = conn.SessionConnection(session_key,r)

In [59]:
reload(conn)

<module 'conn' from 'conn.py'>

In [9]:
print(dc.games_list_key())
print(dc.team_list_key())
print(dc.game_key(1))
print(dc.strength_key(1))
print(dc.property_key(1,'team_1'))
dc.r_conn.lrange(dc.team_list_key(),0,-1)

s:1:game_ids
s:1:t_ids:
s:1:game:1
s:1:strengths:1
s:1:game:1:team_1


[]

In [10]:
def initilize_session(s_id,name,team_ids,r_conn)
    session_key = "".join(["s:",str(s_id)])
    session_name_key = "".join([session_key,'name'])
    r.set(session_name_key,name)
    sc = conn.SessionConnection(session_key,r_conn)
    r_conn.set(sc.new_game_id_key(),0)
    
    for id in team_ids:
        r.rpush(sc.team_list_key(),id)

    


-1

In [176]:
teams = list(set([g.teams[0].id for g in sess.games] + [g.teams[1].id for g in sess.games]))
d = {'a':1,'b':2,'c':3}
d.keys()

['a', 'c', 'b']

In [143]:
r.delete(dc.team_list_key())
r.delete(dc.games_list_key())
test_games = {0:[1,0,1,3,1,0,0,1],
              1:[3,2,1,3,1,0,0,1],
              2:[2,0,1,3,2,0,0,1],
              3:[3,1,1,5,2,0,0,1],
              4:[2,1,0,0,3,0,0,0]}
def make_game(id,props):
    game = dict(zip(['team_1','team_2','score_1','score_2',
                     'round','s_updated','kl_updated','status'],props))
    game['id'] = id
    return game

for k in test_games:
   dc.set_game(make_game(k,test_games[k])) 
   r.rpush(dc.games_list_key(),k)

def set_team_ids(ids):
    for id in ids:
        r.rpush(dc.team_list_key(),id)
    
set_team_ids([0,1,2,3,4])
teams = dc.make_teams(dc.get_team_list())
games = dc.get_games()
sess = ranking.Seeding(teams)
sess.games = dc.mcmc_games(games,teams)
sess.games[4].scores
sess.mc_points=1000
sess.mc_burn = 100
sess.fit_model()

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.7 sec

In [167]:
reload(evaluate)
reload(ranking)
reload(conn)
evaluate.update_results(session_key,r)
evaluate.update_kl_info(session_key,r)


running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.9 secrunning MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.6 sec

0

In [168]:
evaluate.get_best_staging(session_key,r,3)

[[<tournament.Team instance at 0x10b3a8128>, <tournament.Team instance at 0x10b3a8248>], [<tournament.Team instance at 0x10b3a81b8>, <tournament.Team instance at 0x10b3a8200>]]


0

In [152]:
kl_vec = sess.kl_info_vec()
gs = sess.stage_round(2,sess.kl_info_dict(kl_vec))
t0 = gs[0].teams[1]
t0.id

3

In [161]:
dc.set_stage_game(gs[0])

In [157]:
dc.new_game_id()

8

In [77]:
new_game_id_key = "".join([session_key,"new_game_id"])
def new_game_id():
    id = r.get(new_game_id_key)
    r.incr(new_game_id_key)
    return int(id)
r.set(new_game_id_key,0)


True

In [130]:
sess=evaluate.fit_session(dc)
sess.strengths()
strengths = dict(zip([t.id for t in sess.teams],sess.strengths()))
for id in strengths:
    print(id)
    print(dc.strength_key(id))

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.6 sec0
s:1:strengths:0
1
s:1:strengths:1
2
s:1:strengths:2
3
s:1:strengths:3
4
s:1:strengths:4


In [26]:
reload(mock)
reload(ranking)
s = mock.generate_session(8)
s.mc_points = 10000
s.mc_burn = 100
s = mock.generate_round(s,1)
s.fit_model()
s = mock.generate_round(s,2)
s.fit_model()
s = mock.generate_round(s,3)
s.fit_model()

running MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 5.8 secrunning MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 6.0 secrunning MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 6.3 sec

In [28]:
s = generate_session(28)
s.mc_points = 10000
s.mc_burn = 1000
s = generate_round(s,1)
s.fit_model()

%time games = s.update_KL_graph()


running MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 3.4 secCPU times: user 45.2 s, sys: 83.8 ms, total: 45.3 s
Wall time: 45.2 s


In [29]:
s = generate_session(28)
s.mc_points = 100000
s.mc_burn = 10000
s = generate_round(s,1)
s.fit_model()

%time games = s.update_KL_graph()


running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 34.4 secCPU times: user 7min 3s, sys: 182 ms, total: 7min 3s
Wall time: 7min 3s


In [149]:
id = dc.new_game_id()
id

In [142]:
r.flushall()
r.set(dc.new_game_id_key(),0)

True

In [64]:
f = kl_func(s)
%time parmap(f,range(378))
%time s.fit_model()

CPU times: user 933 ms, sys: 121 ms, total: 1.05 s
Wall time: 5min 21s
running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 35.8 secCPU times: user 40.4 s, sys: 27.3 s, total: 1min 7s
Wall time: 35.8 s


In [29]:
pprint([[t.id, t.strength, s.strengths()[ind]] for ind,t in enumerate(s.teams)])
math.sqrt(sum([([t.strength for t in s.teams][ind]-st)**2 for ind,st in enumerate(s.strengths())]))/s.n_teams

[   [0, 0.3752943620190973, 0.45426442959151403],
    [1, -1.6376178714075802, -2.3115184423612858],
    [2, -0.11921177384034226, 0.81934107408149992],
    [3, 0.44828581135608675, 0.054955749685839814],
    [4, 0.6310517958122059, 0.26132250485710684],
    [5, 1.196535761598349, 1.8662282575723685],
    [6, 0.6900951751681348, -0.012526869477989833],
    [7, -1.584433260705951, -1.1320667039490466]]


0.20839675440988437

In [63]:
len(s.marginal_ind)

378

In [24]:
[[[t.id for t in g.teams],g.scores,g.round] for g in s.games]

[[[0, 7], [5, 1], 1],
 [[1, 6], [0, 0], 1],
 [[2, 5], [0, 0], 1],
 [[3, 4], [2, 9], 1]]

In [860]:
pr_f = lambda x: math.exp(s.prob_func(s.teams[3].strength-s.teams[1].strength,1.0,x[0],x[1]))

In [861]:
[discrete_lahiri(pr_f,score_choices(10)) for x in range(20)]

[[5, 2],
 [1, 0],
 [3, 2],
 [3, 4],
 [0, 6],
 [3, 5],
 [0, 6],
 [9, 6],
 [2, 5],
 [9, 9],
 [0, 0],
 [4, 0],
 [8, 6],
 [0, 6],
 [2, 3],
 [6, 6],
 [6, 0],
 [0, 3],
 [8, 9],
 [5, 5]]

In [859]:
s = generate_session(4)
s.scale

2.0

In [391]:
[t.id for t in sess.stage_round(3)[1].teams]

[2, 4]

In [892]:
x = random.choice(score_choices(10))
p = pr_f(x)
u = random.uniform(0,1)
print(x)
print(p)
print(u)

[0, 2]
0.0561291416822
0.787541383464


In [900]:
reload(ranking)
reload(worker)
reload(evaluate)
reload(conn)

<module 'conn' from 'conn.pyc'>

In [918]:
import rq
import worker
import evaluate
import time
import conn
import statistics as stats

In [950]:
queue = rq.Queue(connection=redis.Redis())
s.mc_points=100000
s.mc_burn = 10000
s.fit_model()


running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 80.0 sec

In [520]:
job = queue.enqueue_call(func=evaluate.update_results,args=(sess.games,sess))

In [519]:
job = queue.enqueue(evaluate.test_func)

In [642]:
evaluate.update_results(session_key,r)

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.6 sec

AttributeError: 'int' object has no attribute 'info'

In [740]:
filter(lambda x: not x == 5,range(10))

[0, 1, 2, 3, 4, 6, 7, 8, 9]

In [468]:
print(job.result)

None


In [489]:
queue.get_job_ids()

[u'848e283d-0d6b-47e4-a7a1-9b37899dba0c']

In [524]:
import pickle

In [38]:
r.hset('myhash','t1',1)
r.hset('myhash','t2',2)
r.hset('myhash','val',2.12321)
d = r.hgetall('myhash')
for key,val in d.iteritems():
    d[key] = num(val)
d


{'t1': 1, 't2': 2, 'val': 2.12321}

In [36]:
d={}

0.6029027210959689

<function numpy.core.fromnumeric.mean>

In [39]:
r.flushall()

True